In [1]:
import torch
import torchvision

import cv2
import monai # gradcam 사용하기 위함
import numpy as np
import matplotlib.pyplot as plt

from sklearn import manifold # TSNE 사용
from sklearn.decomposition import PCA # PCA 사용
from sklearn.cluster import KMeans # KMeans 사용

## 2D Image input 확인



In [2]:
def check_input(iterator, nrow): # iterator = dataloader, nrow = plot하고 싶은 열의 개수
    for data in iterator:
        input = data['img']
        label = data['label']
        
        # nrow로 한 행의 열 갯수 정함(행 갯수 : batch_size / nrow), permute(1,2,0) : [C,H,W] -> [H,W,C]
        plt.imshow(torchvision.utils.make_grid(input, nrow=nrow,normalize=True).permute(1,2,0)) 
        print(''.join(f'{label[i]} 'for i in range(input.shape[0]))) # input.shape[0] : batch_size 
        plt.show()

## Hook

In [ ]:
class Hook():
    def __init__(self, module, backward = False): 
        if backward == False:
            self.hook = module.register_forward_hook(self.hook_fn) # 순전파 진행할때의 feature 뽑을때 사용
        else:
            self.hook = module.register_backward_hook(self.hook_fn) # 역전파 진행할때의 feature 뽑을때 사용
    def hook_fn(self, module, input, output): 
        self.input = input # hook.input : hook이 걸린 layer input 값
        self.features = output # hook.feauture : hook이 걸린 layer output 값
    def close(self):
        self.hook.remove() # hook 제거

## Feature extraction

In [3]:
def get_representations(model, iterator, device, hook = None): 
    '''
    If Hook is not None.
        ex)
        hook_model = Hook(model.layer4) 
        def get_representations(model, test_loader, device, hook = hook_model):
    '''
    model.eval()

    outputs = [] 
    labels = [] 

    with torch.no_grad():
        for data in iterator:
            input = data['img'].to(device) 
            label = data['label']
            
            output = model(input) # 모델 마지막 층 output
            
            if hook is not None: # hook이 걸려있다면
                output = hook.features # output : hook이 걸려있는 layer의 output
                
            outputs.append(output.cpu()) # pca, tsne, kmeans 사용하기 위해 gpu -> cpu
            labels.append(label) 
    
    # batch 기준으로 다시 concat 
    outputs = torch.cat(outputs, dim = 0)  
    labels = torch.cat(labels, dim = 0) 
    return outputs, labels

## PCA

In [4]:
def get_pca(data, n_components = 2): 
    pca = PCA(n_components= n_components) # n_components 값으로 차원 축소 설정
    pca_data = pca.fit_transform(data) # 위의 조건 적용
    return pca_data

# TSNE

In [6]:
def get_tsne(data, n_components = 2):
    tsne = manifold.TSNE(n_components = n_components, random_state = 0) # n_components 값으로 차원 축소 설정
    tsne_data = tsne.fit_transform(data) # 위의 조건 적용
    return tsne_data

## Plot Representation(PCA, TSNE)

In [7]:
def plot_representations(data, labels, n_images = None):
    '''
    ex)
    outputs, labels = get_representation(model, test_loader)
    
    output_pca = get_pca(outputs)
    plot_representations(output_pca, labels)
    
    output_tsne = get_tsne(outputs)
    plot_representations(output_tsne, labels)
    '''            
    if n_images is not None: # feature extraction에서 원하는 갯수의 feature만 사용할때
        data = data[:n_images]
        labels = labels[:n_images]
                
    fig = plt.figure(figsize = (15, 15))
    ax = fig.add_subplot(111)
    scatter = ax.scatter(data[:, 0], data[:, 1], c = labels, cmap = 'hsv') # 데이터 라벨별 다른 color를 적용
    plt.show()


## K-Means Clustering

In [ ]:
def plot_kmeans(data, n_cluster, n_images = None):
    '''
    ex)
    outputs, labels = get_representation(model, test_loader)
    plot_kmeans(outputs, n_cluster = 5)
    '''
    if n_images is not None: # feature extraction에서 원하는 갯수의 feature만 사용할때
        data = data[:n_images]
    
    kmeans = KMeans(n_clusters = n_cluster, random_state = 0) # 몇개의 cluster를 만들지 설정
    k_labels = kmeans.fit_predict(data) # 위의 조건 적용

    unique_labels = np.unique(k_labels) # label의 unique 값을 담은 list
    centroids = kmeans.cluster_centers_ # 만들어진 cluster의 center값을 담은 list
    
    plt.figure(figsize = (15, 15))
    for i in unique_labels: 
        # label별로 색을 다르게 해서 하나의 figure에 겹쳐서 뿌림
        plt.scatter(data[k_labels == i, 0], data[k_labels == i, 1], label = i, s = 80, cmap = 'hsv')
    plt.scatter(centroids[:, 0], centroids[:, 1], s = 90, color = 'k', marker='x') # 각 cluster의 center 표시
    plt.show()

## Easy & Difficult 

In [ ]:
def test_easy_difficult(iterator, model, target_layer, device):
    '''
    ex)
    test_easy_difficult(test_loader, model, target_layer = 'layer4', device)
    '''

    cam = monai.visualize.GradCAM(nn_module=model, target_layers= target_layer)

    for data in iterator:
        input = data['img'].to(device)
        label = data['label']

        output = model(input)
        _, pred = torch.max(output, dim=1) # _ : max, pred : max_index binary로 생각하고 만들었음 그때 그때 달라져야함
        # grad cam 결과, [b,c,h,w -> b,h,w,c] : opencv로 colormap 적용하기 위함
        cam_result = cam(input.float().to(device)).permute(0,2,3,1) 

        for i in range(cam_result.shape[0]): # cam_result.shape[0] : batchsize
            # opencv 쓰기 위해 gpu -> cpu, 0~1로 normalize된 이미지를 0~255범위로 변경 후 colormap 적용
            # -> (0 ~ 1)범위인 image와 겹쳐서 표현하기 위해 다시 255로 나눔
            cam_show = cv2.applyColorMap(np.uint8(cam_result[i].detach().cpu().numpy() * 255), cv2.COLORMAP_JET)/255
            
            fig = plt.figure()
            ax1 = fig.add_subplot(121)
            ax1.imshow(input[i].detach().cpu().numpy().transpose(1,2,0), 'gray')
            ax1.axis('off')

            ax2 = fig.add_subplot(122)
            ax2.imshow(input[i].detach().cpu().numpy().transpose(1,2,0), 'gray')
            ax2.imshow(cam_show, alpha = 0.3) # glay scale 이미지에 투명도 0.3인 cam 사진 겹치게 함 
            ax2.axis("off")

            fig.suptitle(f'GT : {int(label[i])}  Pred : {int(pred[i])}') # title에 "GT : label Pred : label" 표시
            fig.tight_layout()
    plt.show()